In [1]:
from lib.network import PoseNetRGBOnly
from lib.loss import Loss
from lib.utils import setup_logger
from datasets.linemod.dataset import PoseDataset as PoseDataset_linemod
import torch
import os
from torch import optim
import numpy as np
import time
from torch.autograd import Variable
%load_ext autoreload
%autoreload 2


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
num_objects = 5
num_points = 500
outf = 'trained_models/linemod'
log_dir = 'experiments/logs/linemod'
repeat_epoch = 20
decay_margin = 0.016
decay_start = False
lr_rate = 0.3
w_rate = 0.3

In [6]:
estimator = PoseNetRGBOnly(num_points=num_points, num_obj = num_objects)
estimator.cuda();
lr = 0.0001
optimizer = optim.Adam(estimator.parameters(), lr=lr)

In [5]:
model = 'trained_models/linemod/pose_model_12_0.09392938120923575.pth'
estimator.load_state_dict(torch.load(model))

FileNotFoundError: [Errno 2] No such file or directory: 'trained_models/linemod/pose_model_12_0.09392938120923575.pth'

In [7]:
dataset_root = "./datasets/linemod/Linemod_preprocessed"
noise_trans = 0.03
refine_start = False
decay_start = False


dataset = PoseDataset_linemod('train', 
                              num_points, 
                              True, 
                              dataset_root, 
                              noise_trans, 
                              refine_start)

workers = 4
dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=1, 
                                         shuffle=True, 
                                         num_workers=workers)

Object 2 buffer loaded
Object 4 buffer loaded
Object 5 buffer loaded
Object 10 buffer loaded
Object 11 buffer loaded


In [8]:
test_dataset = PoseDataset_linemod('test', 
                                   num_points, 
                                   False, 
                                   dataset_root, 
                                   0.0, 
                                   refine_start)
testdataloader = torch.utils.data.DataLoader(test_dataset, 
                                             batch_size=1, 
                                             shuffle=False, 
                                             num_workers=workers)

Object 2 buffer loaded
Object 4 buffer loaded
Object 5 buffer loaded
Object 10 buffer loaded
Object 11 buffer loaded


In [9]:
sym_list = dataset.get_sym_list()
num_points_mesh = dataset.get_num_points_mesh()

In [10]:
print('>>>>>>>>----------Dataset loaded!---------<<<<<<<<\nlength of the training set: {0}\nlength of the testing set: {1}\nnumber of sample points on mesh: {2}\nsymmetry object list: {3}'
      .format(len(dataset), 
              len(test_dataset), 
              num_points_mesh, 
              sym_list))

>>>>>>>>----------Dataset loaded!---------<<<<<<<<
length of the training set: 914
length of the testing set: 515
number of sample points on mesh: 500
symmetry object list: [3, 4]


In [11]:
criterion = Loss(num_points_mesh, sym_list)

In [16]:
start_epoch = 1
nepoch = 16
w = 0.015
batch_size = 8

In [17]:
next(iter(dataloader))[1].shape

torch.Size([1, 1, 500])

In [18]:
lr *= lr_rate
w *= w_rate
optimizer = optim.Adam(estimator.parameters(), lr=lr)

In [19]:
best_test = np.Inf
print_every = 50

if start_epoch == 1:
    for log in os.listdir(log_dir):
        if '.ipyn' not in log:
            os.remove(os.path.join(log_dir, log))
st_time = time.time()

for epoch in range(start_epoch, nepoch):
    logger = setup_logger('epoch%d' % epoch, os.path.join(log_dir, 'epoch_%d_log.txt' % epoch))
    mess = 'Train time {0}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)) + ', ' + 'Training started')
    print(mess)
    logger.info(mess)
    
    train_count = 0
    train_dis_avg = 0.0
    
    estimator.train()
    optimizer.zero_grad()

    for rep in range(repeat_epoch):
        for i, data in enumerate(dataloader, 0):
            points, choose, img, target, model_points, idx = data 
            points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                             Variable(choose).cuda(), \
                                                             Variable(img).cuda(), \
                                                             Variable(target).cuda(), \
                                                             Variable(model_points).cuda(), \
                                                             Variable(idx).cuda()
            
            pred_r, pred_t, pred_c, emb = estimator(img, choose, idx)
            loss, dis, _, _ = criterion(pred_r, pred_t, pred_c, target, 
                                  model_points, idx,
                                  points, w, refine_start)
            loss.backward()
            
            train_dis_avg += dis.item()
            train_count += 1
            if train_count % batch_size == 0:
                
                mess = 'Train time {0} Epoch {1} Batch {2} Frame {3} Avg_dis:{4}'.format(
                        time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), 
                        epoch, int(train_count / batch_size), 
                        train_count, 
                        train_dis_avg / batch_size)
                logger.info(mess)
                
                optimizer.step()
                optimizer.zero_grad()
                train_dis_avg = 0
                
                
                if train_count % print_every == 0:
                    print(mess)
            
            if train_count != 0 and train_count % 500 == 0:                    
                torch.save(estimator.state_dict(), '{0}/pose_model_current.pth'.format(outf))
            
    print('>>>>>>>>----------epoch {0} train finish---------<<<<<<<<'.format(epoch))
    
    test_dis = 0.0
    test_count = 0
    estimator.eval()
    
    for j, data in enumerate(testdataloader, 0):
        points, choose, img, target, model_points, idx = data
        points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                         Variable(choose).cuda(), \
                                                         Variable(img).cuda(), \
                                                         Variable(target).cuda(), \
                                                         Variable(model_points).cuda(), \
                                                         Variable(idx).cuda()
       
        pred_r, pred_t, pred_c, emb = estimator(img, choose, idx)
        _, dis, _, _ = criterion(pred_r, pred_t, pred_c, target,
                           model_points, idx, 
                           points, w, refine_start)
        test_dis += dis.item()
        mess = 'Test time {0} Test Frame No.{1} dis:{2}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), test_count, dis)
        logger.info(mess)
        
        test_count += 1
    test_dis = test_dis / test_count
    mess = 'Test time {0} Epoch {1} TEST FINISH Avg dis: {2}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), epoch, test_dis)
    print(mess)
    logger.info(mess)
    
    if test_dis <= best_test:
        best_test = test_dis
        torch.save(estimator.state_dict(), '{0}/pose_model_{1}_{2}.pth'.format(outf, epoch, test_dis))
        print(epoch, '>>>>>>>>----------BEST TEST MODEL SAVED---------<<<<<<<<')
        
    if best_test < decay_margin and not decay_start:
        decay_start = True
        lr *= lr_rate
        w *= w_rate
        optimizer = optim.Adam(estimator.parameters(), lr=lr)

Test time 00h 33m 42s Epoch 2 TEST FINISH Avg dis: 0.01673948421206289
2 >>>>>>>>----------BEST TEST MODEL SAVED---------<<<<<<<<
Train time 00h 33m 42s, Training started
Train time 00h 33m 53s Epoch 3 Batch 25 Frame 200 Avg_dis:0.018506382941268384
Train time 00h 34m 04s Epoch 3 Batch 50 Frame 400 Avg_dis:0.013413167383987457
Train time 00h 34m 15s Epoch 3 Batch 75 Frame 600 Avg_dis:0.012544608442112803
Train time 00h 34m 26s Epoch 3 Batch 100 Frame 800 Avg_dis:0.01687428425066173
Train time 00h 34m 37s Epoch 3 Batch 125 Frame 1000 Avg_dis:0.009262320643756539
Train time 00h 34m 48s Epoch 3 Batch 150 Frame 1200 Avg_dis:0.01629018585663289
Train time 00h 34m 59s Epoch 3 Batch 175 Frame 1400 Avg_dis:0.00921548210317269
Train time 00h 35m 10s Epoch 3 Batch 200 Frame 1600 Avg_dis:0.009264854132197797
Train time 00h 35m 21s Epoch 3 Batch 225 Frame 1800 Avg_dis:0.010205122991465032
Train time 00h 35m 32s Epoch 3 Batch 250 Frame 2000 Avg_dis:0.022535425494425
Train time 00h 35m 43s Epoch 3 B

KeyboardInterrupt: 

In [ ]:
t = next(iter(dataloader))

In [14]:
points, choose, img, target, model_points, idx = t

In [15]:
points.shape

torch.Size([1, 500, 3])

In [20]:
points

tensor([[[-0.0631, -0.1219,  1.0574],
         [-0.0650, -0.1200,  1.0574],
         [-0.0670, -0.1185,  1.0604],
         ...,
         [-0.0254, -0.0375,  1.0414],
         [-0.0217, -0.0376,  1.0444],
         [-0.0199, -0.0376,  1.0444]]])

In [23]:
target.shape

torch.Size([1, 500, 3])

In [24]:
choose.shape

torch.Size([1, 1, 500])

In [28]:
model_points.shape

torch.Size([1, 500, 3])

In [29]:
model_points

tensor([[[-0.0658, -0.0487,  0.0008],
         [-0.0638, -0.0188,  0.0085],
         [-0.0629, -0.0243,  0.0079],
         ...,
         [-0.0101,  0.0565, -0.0162],
         [-0.0658,  0.0009,  0.0301],
         [ 0.0184,  0.0379,  0.0174]]])

In [26]:
img.shape

torch.Size([1, 3, 120, 120])

In [27]:
choose

tensor([[[ 1409,  1490,  1495,  1528,  1530,  1638,  1646,  1756,  1770,  1771,
           1855,  1879,  1882,  1885,  1965,  2001,  2016,  2089,  2094,  2098,
           2103,  2105,  2121,  2122,  2134,  2186,  2199,  2200,  2223,  2227,
           2235,  2238,  2240,  2249,  2314,  2325,  2340,  2352,  2362,  2373,
           2425,  2432,  2438,  2452,  2454,  2457,  2474,  2497,  2575,  2583,
           2591,  2607,  2618,  2665,  2674,  2691,  2696,  2712,  2784,  2785,
           2797,  2826,  2852,  2906,  2921,  2924,  2949,  2973,  3022,  3032,
           3046,  3052,  3070,  3078,  3080,  3097,  3142,  3145,  3146,  3147,
           3155,  3159,  3162,  3182,  3201,  3203,  3204,  3219,  3257,  3269,
           3275,  3280,  3292,  3305,  3313,  3319,  3320,  3324,  3329,  3330,
           3340,  3342,  3397,  3408,  3412,  3437,  3448,  3455,  3522,  3523,
           3526,  3536,  3539,  3541,  3542,  3545,  3548,  3561,  3582,  3622,
           3625,  3628,  3638,  3663,  3